In [ ]:
!pip install --upgrade pip
!pip -q install sagemaker awscli boto3 pandas --upgrade 

## Example: PyTorch deployments using TorchServe and Amazon SageMaker

In this example, we’ll show you how you can build a TorchServe container and host it using Amazon SageMaker. With Amazon SageMaker hosting you get a fully-managed hosting experience. Just specify the type of instance, and the maximum and minimum number desired, and SageMaker takes care of the rest.

With a few lines of code, you can ask Amazon SageMaker to launch the instances, download your model from Amazon S3 to your TorchServe container, and set up the secure HTTPS endpoint for your application. On the client side, get prediction with a simple API call to this secure endpoint backed by TorchServe.

Code, configuration files, Jupyter notebooks and Dockerfiles used in this example are available here:
https://github.com/shashankprasanna/torchserve-examples.git


### Clone the TorchServe repository and install torch-model-archiver

You'll use `torch-model-archiver` to create a model archive file. The .mar model archive file contains model checkpoints along with it’s `state_dict` (dictionary object that maps each layer to its parameter tensor).

In [1]:
!git clone https://github.com/pytorch/serve.git
!pip install serve/model-archiver/

fatal: destination path 'serve' already exists and is not an empty directory.
Processing ./serve/model-archiver
  Created wheel for torch-model-archiver: filename=torch_model_archiver-0.1.0b20200417-py3-none-any.whl size=15843 sha256=a54b6d7e88c0942ee055859b9e512772e59f1521446475a7719657d21a7c62e8
  Stored in directory: /home/ec2-user/.cache/pip/wheels/19/d4/31/7884ea1291df5d92cf41bc1e2a78285b61474f93496a7e78af
Successfully built torch-model-archiver
  Attempting uninstall: torch-model-archiver
    Found existing installation: torch-model-archiver 0.1.0b20200417
    Uninstalling torch-model-archiver-0.1.0b20200417:
      Successfully uninstalled torch-model-archiver-0.1.0b20200417


### Download a PyTorch model and create a TorchServe archive

In [2]:
!wget -q https://download.pytorch.org/models/densenet161-8d451a50.pth
    
model_file_name = 'densenet161'

!torch-model-archiver --model-name {model_file_name} \
--version 1.0 --model-file serve/examples/image_classifier/densenet_161/model.py \
--serialized-file densenet161-8d451a50.pth \
--extra-files serve/examples/image_classifier/index_to_name.json \
--handler image_classifier

!ls *.mar

ERROR - /home/ec2-user/SageMaker/test/torchserve-examples/densenet161.mar already exists.
Please specify --force/-f option to overwrite the model archive output file.
See -h/--help for more details.
densenet161.mar


### Upload the generated densenet161.mar archive file to Amazon S3
Choose a unique bucket name. The following lines of code will first create a bucket for you if bucket with the specified name doesn't already exist. Then it'll create a compressed tar.gz file our of the densenet161.mar file. Amazon SageMaker expects that models are in a tar.gz file. Finally it uploads the model to your S3 bucket under the models directory

### Create a boto3 session and get specify a role with SageMaker access

In [3]:
import boto3, time, json
sess    = boto3.Session()
sm      = sess.client('sagemaker')
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [4]:
import sagemaker
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

In [5]:
bucket_name = sagemaker_session.default_bucket()
prefix = 'torchserve'

!tar cvfz {model_file_name}.tar.gz densenet161.mar
!aws s3 cp {model_file_name}.tar.gz s3://{bucket_name}/{prefix}/model

densenet161.mar
upload: ./densenet161.tar.gz to s3://sagemaker-us-west-2-453691756499/torchserve/model


### Create an Amazon ECR registry
Create a new docker container registry for your torchserve container images.

In [6]:
registry_name = 'torchserve'
!aws ecr create-repository --repository-name {registry_name}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'torchserve' already exists in the registry with id '453691756499'


### Build a TorchServe Docker container and push it to Amazon ECR

In [8]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}

Sending build context to Docker daemon  510.3MB
Step 1/16 : FROM ubuntu:18.04
 ---> 4e5021d210f6
Step 2/16 : ENV PYTHONUNBUFFERED TRUE
 ---> Using cache
 ---> 9606516f46f1
Step 3/16 : RUN apt-get update &&     DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y     fakeroot     ca-certificates     dpkg-dev     g++     python3-dev     openjdk-11-jdk     curl     vim     && rm -rf /var/lib/apt/lists/*     && cd /tmp     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> f06b77639d7e
Step 4/16 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
 ---> Using cache
 ---> 90d7baec3c67
Step 5/16 : RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
 ---> Using cache
 ---> b9ed1cdff896
Step 6/16 : RUN pip install --no-cache-dir psutil                 --no-cache-dir torch                 --no-cache-dir torchvision
 ---> Using cache
 ---> 3a49828db3b7
Step 7/16 : ADD serve ser

### Deploy endpoint and make prediction using Amazon SageMaker SDK

In [9]:
from sagemaker.model import Model
from sagemaker.predictor import RealTimePredictor

model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name}.tar.gz'
sm_model_name = 'torchserve-densenet161'

torchserve_model = Model(model_data = model_data, 
                         image = image,
                         role  = role,
                         predictor_cls=RealTimePredictor,
                         name  = sm_model_name)

In [ ]:
endpoint_name = 'torchserve-endpoint-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = torchserve_model.deploy(instance_type='ml.m4.xlarge',
                                    initial_instance_count=1,
                                    endpoint_name = endpoint_name)

Using already existing model: torchserve-densenet161


-------

#### Test the TorchServe hosted model

In [ ]:
!wget -q https://s3.amazonaws.com/model-server/inputs/kitten.jpg    
file_name = 'kitten.jpg'
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = payload
    
response = predictor.predict(data=payload)
print(*json.loads(response), sep = '\n')

### Deploy endpoint and make prediction using Python SDK (Boto3)

In [ ]:
model_data = f's3://{bucket_name}/{prefix}/models/{model_file_name}.tar.gz'
sm_model_name = 'torchserve-densenet161-boto'

container = {
    'Image': image,
    'ModelDataUrl': model_data
}

create_model_response = sm.create_model(
    ModelName         = sm_model_name,
    ExecutionRoleArn  = role,
    PrimaryContainer  = container)

print(create_model_response['ModelArn'])

In [ ]:
import time
endpoint_config_name = 'torchserve-endpoint-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants = [{
        'InstanceType'        : 'ml.m4.xlarge',
        'InitialVariantWeight': 1,
        'InitialInstanceCount': 1,
        'ModelName'           : sm_model_name,
        'VariantName'         : 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [ ]:
endpoint_name = 'torchserve-endpoint-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(endpoint_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName         = endpoint_name,
    EndpointConfigName   = endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

In [ ]:
resp = sm.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

In [ ]:
!wget https://s3.amazonaws.com/model-server/inputs/kitten.jpg    
file_name = 'dog.jpg'
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = payload

In [ ]:
import json
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)

print(*json.loads(response['Body'].read()), sep = '\n')